<a href="https://colab.research.google.com/github/dokeyuka/deep-learning-practice/blob/main/deep_learning_practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 ＭＬＰのPyTorchによる実装

In [5]:
import numpy as np
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optimizers


'''
  (1) modelの実装
'''
class MLP(nn.Module):
  '''
  多層パーセプトロン
  '''
  def __init__(self, input_dim, hidden_dim, output_dim):

    #nn,Moduleのコンストラクタをオーバーライド。親クラスの__init__と同じ処理をしてくれる
    super().__init__()

    #nn.Linearとは別に活性化関数の`層’を追加
    self.l1 = nn.Linear(input_dim, hidden_dim)
    self.a1 = nn.Sigmoid()
    self.l2 = nn.Linear(hidden_dim, output_dim)
    self.a2 = nn.Sigmoid()


    #self.layersで層をすべてまとめる
    self.layers = [self.l1, self.a1, self.l2, self.a2]

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

if __name__ == '__main__':
  np.random.seed(123)
  #torch用の乱数シード
  torch.manual_seed(123)

  #deviceに実行環境を格納して同じコードでCPUでもGPUでも対応できるように
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  '''
  1 データの準備
  '''
  N = 300
  x, t = datasets.make_moons(N, noise = 0.3)
  t = t.reshape(N,1)
  x_train, x_test ,t_train, t_test = train_test_split(x, t, test_size = 0.2)

  
  '''
  2 modelの構築
  '''
  #.to(device)でGPUにモデルを対応させる
  model = MLP(2, 3, 1).to(device)


  '''
  3 modelの学習
  '''
  #2クラス交差エントロピー誤差関数
  criterion = nn.BCELoss()
  #最適化optimizeしたいパラメータを引数に与える
  #lr = learning rate
  optimizer = optimizers.SGD(model.parameters(), lr = 0.1)

 
 
  def compute_loss(t, y):
    return criterion(y, t)

  #勾配降下法で誤差の計算
  def train_step(x,t):
    #モデルを学習モードにする
    model.train()
    
    preds = model(x)
    loss = compute_loss(t, preds)
    #勾配計算
    optimizer.zero_grad()
    #誤差逆伝搬
    loss.backward()
    #parameterの更新
    optimizer.step()
    return loss


#ミニバッチ学習
  epochs = 100
  batch_size = 10
  n_batches = x_train.shape[0] // batch_size

  for epoch in range(epochs):
    train_loss = 0.
    x_, t_ = shuffle(x_train, t_train)

    #pytorchでは明示的にtorch.Tensorに変換すべし
    x_ = torch.Tensor(x_).to(device)
    t_ = torch.Tensor(t_).to(device)

    for n_batch in range(n_batches):
      start = n_batch * batch_size
      end = start + batch_size
      #
      loss = train_step(x_[start: end], t_[start: end])
      #loss.item() で誤差の値をnumpyに変換
      train_loss += loss.item()
    
    print('epoch: {}, loss: {:.3}'.format(epoch + 1, train_loss))
  
  '''
  4 モデルの評価
  '''
  def test_step(x, t):
    x = torch.Tensor(x).to(device)
    t = torch.Tensor(t).to(device)
    #modelを推論モードに切り替え
    model.eval()
    
    preds = model(x)
    loss = compute_loss(t, preds)
    return loss, preds

  loss, preds = test_step(x_test, t_test)
  #loss とpredictionをnumpy配列に変換
  test_loss = loss.item()
  preds = preds.data.cpu().numpy() > 0.5
  test_acc = accuracy_score(t_test, preds)

  print('test_loss:{:.3f}, test_acc:{:.3f}'.format(test_loss, test_acc))



epoch: 1, loss: 17.1
epoch: 2, loss: 16.8
epoch: 3, loss: 16.6
epoch: 4, loss: 16.4
epoch: 5, loss: 16.1
epoch: 6, loss: 15.9
epoch: 7, loss: 15.6
epoch: 8, loss: 15.3
epoch: 9, loss: 14.9
epoch: 10, loss: 14.5
epoch: 11, loss: 14.1
epoch: 12, loss: 13.7
epoch: 13, loss: 13.3
epoch: 14, loss: 12.9
epoch: 15, loss: 12.5
epoch: 16, loss: 12.2
epoch: 17, loss: 11.8
epoch: 18, loss: 11.6
epoch: 19, loss: 11.3
epoch: 20, loss: 11.1
epoch: 21, loss: 10.9
epoch: 22, loss: 10.8
epoch: 23, loss: 10.6
epoch: 24, loss: 10.5
epoch: 25, loss: 10.4
epoch: 26, loss: 10.3
epoch: 27, loss: 10.2
epoch: 28, loss: 10.1
epoch: 29, loss: 10.1
epoch: 30, loss: 10.0
epoch: 31, loss: 9.96
epoch: 32, loss: 9.92
epoch: 33, loss: 9.86
epoch: 34, loss: 9.82
epoch: 35, loss: 9.79
epoch: 36, loss: 9.75
epoch: 37, loss: 9.73
epoch: 38, loss: 9.71
epoch: 39, loss: 9.68
epoch: 40, loss: 9.64
epoch: 41, loss: 9.63
epoch: 42, loss: 9.61
epoch: 43, loss: 9.6
epoch: 44, loss: 9.56
epoch: 45, loss: 9.55
epoch: 46, loss: 9.5

# 2 MNISTdataを使ったPytorchによる実装

In [ ]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn 
import torch.optim as optimizers
from torchvision import datasets
import torchvision.transforms as transforms

'''
  (1) modelの実装
'''
'''
  1 データの準備
'''
'''
  2 モデルの構築
'''
'''
  3　モデルの学習
'''
'''
  4 モデルの評価
'''